<a href="https://colab.research.google.com/github/Paolo20224/Prueba/blob/main/Simulacion_Examen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>